# Logistic Regression - Churn Analysis on Marketing Clients

### Importing required packages

In [236]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregdoc').getOrCreate()

In [237]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pandas as pd

## Old (training) data set from marketing company

### Reading Data

In [238]:
data = spark.read.csv('gs://dataproc-staging-us-central1-53720095629-mubhdkfd/notebooks/jupyter/customer_churn.csv',inferSchema=True,header=True)

data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [239]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Data Exploration

In [240]:
data.select('Company').show()

+--------------------+
|             Company|
+--------------------+
|          Harvey LLC|
|          Wilson PLC|
|Miller, Johnson a...|
|           Smith Inc|
|          Love-Jones|
|        Kelly-Warren|
|   Reynolds-Sheppard|
|          Singh-Cole|
|           Lopez PLC|
|       Reed-Martinez|
|Briggs, Lamb and ...|
|    Figueroa-Maynard|
|     Abbott-Thompson|
|Smith, Kim and Ma...|
|Snyder, Lee and M...|
|      Sanders-Pierce|
|Andrews, Adams an...|
|Morgan, Phillips ...|
|      Villanueva LLC|
|Berry, Orr and Ca...|
+--------------------+
only showing top 20 rows



In [241]:
#potential categorical variable to see if there are different departments at the same companies
data.select('Company').distinct().count()

873

In [242]:
#most companies are unique so likely not a helpful column
data.select('Company').count()

900

In [243]:
data.select('Num_Sites').show()

+---------+
|Num_Sites|
+---------+
|      8.0|
|     11.0|
|     12.0|
|     10.0|
|      9.0|
|      8.0|
|     11.0|
|      9.0|
|     11.0|
|     11.0|
|      8.0|
|     11.0|
|     11.0|
|     13.0|
|     11.0|
|     12.0|
|     11.0|
|     12.0|
|      8.0|
|      8.0|
+---------+
only showing top 20 rows



In [244]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

### Final column selection

In [245]:
my_cols = data.select([
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Churn'])

In [246]:
#checking row count prior to dropping
my_cols.count()

900

In [247]:
#dropping rows
my_final_data = my_cols.na.drop()
my_final_data.count()

900

### Preprocessing 

In [248]:
#list of independent variables
col_names = [
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'
]

In [249]:
assembler = VectorAssembler(
    inputCols=col_names
    ,outputCol='features'
)

### Model build

In [268]:
log_reg_marketing = LogisticRegression(featuresCol='features',labelCol='Churn')

In [269]:
pipeline = Pipeline(stages=[assembler,log_reg_marketing])

In [270]:
#Using a high train split % becasue we have another data set
train_marketing_data, test_marketing_data = my_final_data.randomSplit([0.8,.2])

In [271]:
#fitting on traingingd data
fit_model = pipeline.fit(train_marketing_data)

In [272]:
import numpy as np
#examining coefficients by creating a pandas dataframe to view stages data
df_coeff = pd.DataFrame(columns=['Variables', 'Coefficients'])
stages = fit_model.stages[-1]
df_coeff.Variables = col_names
df_coeff.Coefficients = np.exp(stages.coefficients)


## number of sites seems to be the most likely predictor of churn
df_coeff

,Variables,Coefficients
0,Age,1.066483
1,Total_Purchase,1.000083
2,Account_Manager,1.787505
3,Years,1.749076
4,Num_Sites,3.276756


In [273]:
#predicting on test data
results = fit_model.transform(test_marketing_data)

In [274]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [275]:
results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [276]:
AUC = my_eval.evaluate(results)
AUC

0.7268315018315018

## New (Test) data set from marketing company

In [283]:
#retrieving data
data_new = spark.read.csv('gs://dataproc-staging-us-central1-53720095629-mubhdkfd/notebooks/jupyter/new_customers.csv',inferSchema=True,header=True)

#selecting data
my_cols_new = data_new.select([
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'
])


#dropping null rows
my_final_data_new = my_cols_new.na.drop()



##### Preprocessing #####
#identifying independent variables
assembler = VectorAssembler(
    inputCols=col_names
    ,outputCol='features'
)

#identifying dependent variable 
log_reg_marketing = LogisticRegression(featuresCol='features',labelCol='Churn')

##### Building Model #####
pipeline = Pipeline(stages=[assembler,log_reg_marketing])

##### Prediciting #####
results = fit_model.transform(my_final_data_new)

results.show()

+----+--------------+---------------+-----+---------+--------------------+--------------------+--------------------+----------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|            features|       rawPrediction|         probability|prediction|
+----+--------------+---------------+-----+---------+--------------------+--------------------+--------------------+----------+
|37.0|       9935.53|              1| 7.71|      8.0|[37.0,9935.53,1.0...|[2.20948501564485...|[0.90109804070902...|       0.0|
|23.0|       7526.94|              1| 9.28|     15.0|[23.0,7526.94,1.0...|[-5.8756903966012...|[0.00279899929473...|       1.0|
|65.0|         100.0|              1|  1.0|     15.0|[65.0,100.0,1.0,1...|[-3.3348524946097...|[0.03439470601901...|       1.0|
|32.0|        6487.5|              0|  9.4|     14.0|[32.0,6487.5,0.0,...|[-4.6683336686112...|[0.00930058687478...|       1.0|
|32.0|      13147.71|              1| 10.0|      8.0|[32.0,13147.71,1....|[0.98502554655003...|[0.728104